# Spark DataFrame Basics

## Part One

In [1]:
import findspark
findspark.init("/home/rodolfo/spark-3.3.1-bin-hadoop3")
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("Basics").getOrCreate()

22/11/27 14:05:18 WARN Utils: Your hostname, rodolfo-300E5M-300E5L resolves to a loopback address: 127.0.1.1; using 10.0.0.107 instead (on interface wlp3s0)
22/11/27 14:05:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/27 14:05:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.json("../../data/people.json")

In [4]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [5]:
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [6]:
df.columns

['age', 'name']

In [8]:
df.describe().show()

+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 2|      3|
|   mean|              24.5|   null|
| stddev|7.7781745930520225|   null|
|    min|                19|   Andy|
|    max|                30|Michael|
+-------+------------------+-------+



In [9]:
from pyspark.sql.types import StructField, StructType, IntegerType, StringType

In [10]:
data_schema = [
    StructField(
        "age",
        IntegerType(),
        True
    ),
    StructField(
        "name",
        StringType(),
        True
    )
]

In [11]:
final_struct = StructType(
    fields=data_schema
)

In [12]:
df = spark.read.json("../../data/people.json", schema=final_struct)

In [13]:
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



## Part Two

In [16]:
type(df["age"])

pyspark.sql.column.Column

In [18]:
df.select("age").show()

+----+
| age|
+----+
|null|
|  30|
|  19|
+----+



In [19]:
type(df.select("age"))

pyspark.sql.dataframe.DataFrame

In [22]:
type(df.head(2)[0])

pyspark.sql.types.Row

In [23]:
df.select(["age", "name"]).show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



### Creating new columns

In [26]:
df.withColumn("double_age", df["age"]*2).show()

+----+-------+----------+
| age|   name|double_age|
+----+-------+----------+
|null|Michael|      null|
|  30|   Andy|        60|
|  19| Justin|        38|
+----+-------+----------+



Modifications with withColumns don't happen inplace on df

In [27]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



### Renaming columns

In [28]:
df.withColumnRenamed("age", "my_new_age").show()

+----------+-------+
|my_new_age|   name|
+----------+-------+
|      null|Michael|
|        30|   Andy|
|        19| Justin|
+----------+-------+



### SQL queries

Start creating temporary view

In [38]:
df.createOrReplaceTempView("people")

In [39]:
results = spark.sql("SELECT * FROM people")

In [40]:
results.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [41]:
results = spark.sql("SELECT * FROM people WHERE age=30")

In [42]:
results.show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+

